In [ ]:
#!pip install langchain_core

In [ ]:
#!pip install langgraph

In [ ]:
#!pip install "generative-ai-hub-sdk[all]" 

# AI agent leveraging SAP Generative AI Hub

### Get credentials for supporting systems

Read all credentials

In [ ]:
import json
with open('./credentials.json', 'r') as creds:
  credentials = json.load(creds)

Credentials for SAP AI Core (to access Large Language Models on the SAP Generative AI Hub)

In [ ]:
import os
os.environ["AICORE_CLIENT_ID"]      = credentials["SAP_AI_CORE"]["AICORE_CLIENT_ID"]
os.environ["AICORE_CLIENT_SECRET"]  = credentials["SAP_AI_CORE"]["AICORE_CLIENT_SECRET"]
os.environ["AICORE_AUTH_URL"]       = credentials["SAP_AI_CORE"]["AICORE_AUTH_URL"]
os.environ["AICORE_RESOURCE_GROUP"] = credentials["SAP_AI_CORE"]["AICORE_RESOURCE_GROUP"]
os.environ["AICORE_BASE_URL"]       = credentials["SAP_AI_CORE"]["AICORE_BASE_URL"]   

Credentials for SMTP server (to send emails)

In [ ]:
smtp_server   = credentials["SMTP"]["SMTP_SERVER"]
smtp_port     = credentials["SMTP"]["SMTP_PORT"]
smtp_user     = credentials["SMTP"]["SMTP_USER"]
smtp_password = credentials["SMTP"]["SMTP_PASSWORD"]

### Define the tools the AI agent can leverage

Each tool is provided as an independent function. The comments in the functions will be used by the AI agent to understand the functionality of each tool, the parameters it might require and which output it returns.

In [ ]:
import random
def get_invoice_status(invoice_id: str) -> str:
   """Returns an invoice's status, ie whether it has been paid or not.

   Args:
      invoice_id: The invoide id
   """

   # This function mocks retrieving the invoice status from a live system
   # See SAP's API documentation for the real / live API that can provide this information from your system, ie the InvoiceClearingStatus on
   # https://help.sap.com/docs/SAP_S4HANA_ON-PREMISE/19d48293097f4a2589433856b034dfa5/cb3caf09bd6749c59f0765981032b74e.html?locale=en-US
   options = ['Paid', 'Overdue', 'Unpaid, not due yet']
   invoice_status = random.choice(options)
   response = f"The status of invoice {invoice_id} is: {invoice_status}." 
    
   return response    

In [ ]:
def get_email_address(name: str) -> str:
   """Returns the person's email address

   Args:
      name: The name of the person whose email address is returned
   """

   # This function mocks retrieving an email address from a live system
   dict = {}
   dict['Ewald'] = 'enteryourownemail@yourcompany.com.xyz'
   dict['Stefan'] = 'enteryourownemail@yourcompany.com.xyz'
   dict['Fabian'] = 'enteryourownemail@yourcompany.com.xyz'

   if name in dict.keys():
      response = dict[name]
   else:
      response = dict['Andreas']
   return response    

In [ ]:
import smtplib, ssl
from email.mime.text import MIMEText
def send_email(recipient_name: str, email_address: str, email_text: str) -> str:
   """Sends emails. Returns a status update.

   Args:
      recipient_name: The name of the email recipient
      email_address: The recipient's email address
      email_text: The email's text that will be send
   """
    
   # This function uses SMTP credentials to send an email directly from Python
   # For productive use you may want to leverage a BTP component, for instance the Alert Notification service
   # https://help.sap.com/docs/alert-notification/sap-alert-notification-for-sap-btp/email-action-type?locale=en-US
    
   # Prepare email content
   subject = 'Email from your SAP BTP AI Agent'
   content = email_text
   msg = MIMEText(content, 'plain')
   msg['Subject'] =  subject
   msg['From']   = smtp_user 

   # Send the email
   context = ssl.create_default_context()  
   with smtplib.SMTP_SSL(smtp_server, smtp_port, context=context) as server:
      server.login(smtp_user, smtp_password)
      server.sendmail(from_addr=smtp_user, to_addrs=email_address, msg=msg.as_string())

   return f"I sent the email to {recipient_name} ({email_address}): \n{email_text}"

In [ ]:
### TODO Requires the implementation of the FAQ assistant as described in:
### https://community.sap.com/t5/artificial-intelligence-and-machine-learning-blogs/hands-on-tutorial-creating-an-faq-assistant-as-tool-for-a-btp-ai-agent/ba-p/14027300

import requests
def answer_SAP_question(text: str) -> str:
   """Responds to questions about the company SAP

   Args:
      text: The question about SAP
   """

   # Forward the incoming question to the AI assistant's REST-API
   backend_api = "https://ENTERYOURURLFROMTHEAIASSISTANT/"
   user_input = text
   paylod = {'user_request': user_input}
   headers = {'Accept' : 'application/json', 'Content-Type' : 'application/json'}
   r = requests.get(backend_api, json=paylod, headers=headers, verify=False)
   response = r.json()

   # Obtain the response and response log
   faq_response = response['faq_response']
   faq_response_log = response['faq_response_log']

   # Return the response    
   return faq_response

In [ ]:
import requests
from bs4 import BeautifulSoup
def get_text_from_link(link: str) -> str:
   """Returns the text of the given website link

   Args:
      link: The link of the webiste whose text is to be returned
   """
    
   # Header to get automated access to SAP's website content
   headers = {
      'referer': 'https://www.scrapingcourse.com/ecommerce/',
      'accept-language': 'en-US,en;q=0.9',
      'content-type': 'application/json',
      'accept-encoding': 'gzip, deflate, br',
      'sec-ch-device-memory': '8',
      'sec-ch-ua': '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
      'sec-ch-ua-platform': "Windows",
      'sec-ch-ua-platform-version': '"10.0.0"',
      'sec-ch-viewport-width': '792'
   }
   # Obtain the website content
   response = requests.get(link, headers=headers)

   # Simple parsing to get the website's text
   soup = BeautifulSoup(response.content, 'html.parser')
   link_text = soup.body.get_text()
   link_text = link_text.replace('\n', '')
   link_text = link_text.replace('Javascript must be enabled for the correct page display', '')

   return link_text

In [ ]:
import requests
from datetime import datetime
def get_lunch_menu() -> str:
   """Returns today's menu of the SAP canteen in Zurich
   """
    
   # Scrape the canteen's website
   response = requests.get("https://circle.sv-restaurant.ch/de/menuplan/chreis-14/")
   soup = BeautifulSoup(response.content, 'html.parser')
    
   # Get current day of week
   dt = datetime.now()
   weekday_current = dt.weekday()
    
   # If called on weekend, use Monday instead
   if weekday_current < 5:
       weekday_menu = weekday_current
   else:
       weekday_menu = 0
    
   # Get date for which menu will be returned
   dates_raw = soup.find_all(class_='date')
   dates = []
   for day in dates_raw:
       dates.append(day.text)
   date = dates[0] # Past dates are removed from the restaurant page
    
   # Get menus for that date
   menus = []
   menus_raw = dates_raw = soup.find_all(id='menu-plan-tab' + str(weekday_menu))
   menus_all_raw = soup.find(id='menu-plan-tab1')
   menus_all = menus_all_raw.find_all(class_='menu-title')
   for menu in menus_all:
       if menu.text not in ['Lunch auf der Terrasse']:
           menus.append(menu.text)
           
   # Prepare the response with the above information
   menu_flowtext = ''
   for i in range(len(menus)):
       menu_flowtext += " " + str(i+1) + ") " + menus[i]
   menu_flowtext = menu_flowtext.lstrip()    
   response = f"On {weekday_menu}, the {date}, Chreis 14 serves {menu_flowtext}."

   return response

In [ ]:
import requests
def get_live_tv_arte() -> str:
   """Returns what is currently shown on the TV stastion ARTE
   """
    
   response = requests.get('https://api.arte.tv/api/player/v2/config/de/LIVE')
   data = response.json()
   title = data['data']['attributes']['metadata']['title']
   description = data['data']['attributes']['metadata']['description']

   return title + ': ' + description

Collect the names of the functions the AI agent can leverage in a list

In [ ]:
tools = [get_invoice_status, get_email_address, send_email, answer_SAP_question, get_text_from_link, get_lunch_menu, get_live_tv_arte]

### Create the AI agent

In [ ]:
from langchain_core.messages import SystemMessage
from langgraph.graph import START, StateGraph, MessagesState
from langgraph.prebuilt import tools_condition, ToolNode
from gen_ai_hub.proxy.langchain.init_models import init_llm
import urllib3
urllib3.disable_warnings()

Initialise the Large Language Model and make the list of tools available to the AI agent

In [ ]:
llm = init_llm('anthropic--claude-3.5-sonnet', max_tokens=300)
llm_with_tools = llm.bind_tools(tools)

Configure the AI agent's behaviour with a system message

In [ ]:
sys_msg = SystemMessage(content="You are a helfpul assistant tasked with answering questions about different topics. Your name is 'SAP BTP AI Agent'. Keep your answers short. After giving a response, do not ask for additional requests. Instead of referring to a link on your response call the function get_text_from_link to get the information from a given link yourself. Only use information that is provided to you by the different tools you are given. When sending email include a greeting and a salutation.")

Define the Assistant node, which leverages the Large Language model, the bound tools and the system message as basis of the AI agent.

In [ ]:
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

Build and compile the graph, which defines the AI agent's capabilities. For full flexibility all tools are connected directly to the initial assistant node. Alternatively tools could be specifically assigned and dedicated as subcomponents to other tools. 

In [ ]:
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
   "assistant",
   # If the latest message (result) from assistant is a tool call -> tools_condition routes to Tools
   # If the latest message (result) from assistant is not a tool call -> tools_condition routes to END
   tools_condition,
)
builder.add_edge("tools", "assistant")
graph = builder.compile()

Visualise the graph

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

### Use the graph with user input

Trigger the graph and receive the initial detailed log

In [ ]:
user_input = "Has invoice 42 been paid?" 
#user_input = "Email Andreas the status of invoice 43" 
#user_input = "Who is the boss at SAP?" 
#user_input = "Should I go for lunch or watch tv?"
agent_outcome = graph.invoke({"messages": [("user", user_input)]})
print(agent_outcome['messages'][-1].to_json()['kwargs']['content'])

Retrieve most important information from the detailed log

In [ ]:
messages_extract = []
for msg in agent_outcome['messages']:
    msg_actor = type(msg).__name__
    msg_text = msg.content
    if msg_actor == 'AIMessage':
        if msg_text == '':
           msg_text = msg.tool_calls
    if msg_actor == 'ToolMessage':
        msg_actor = msg_actor + ' (' + msg.name + ')'
    messages_extract.append([msg_actor, msg_text])
print(str(messages_extract).replace('],', '],\n'))